In [ ]:
!pip install datasets
!pip install torch torchvision torchaudio
!pip install rouge-score nltk
!pip install sentencepiece #что такое sentencepiece?
!pip install evaluate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=5480b192bf30b69080c572fdeb4d53f9a9e84ee543d701783ae49b5340d313aa
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 7.7 MB/s eta 0:00:00


Подробнее о sentencepiece: https://github.com/google/sentencepiece

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import accelerate

accelerate.__version__

'0.25.0'

In [ ]:
from datasets import load_dataset, load_metric
from transformers import (
    DataCollatorForSeq2Seq,
    MT5ForConditionalGeneration,
    MT5Tokenizer,
    pipeline,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
import evaluate
import numpy as np
import datasets

In [ ]:
from datasets import list_datasets
list_datasets()

In [ ]:
#узнайте подробнее о модели mT5
#в чем вообще особенность модели Т5?

tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")  # cointegrated/rut5-small google/mt5-small
model = MT5ForConditionalGeneration.from_pretrained('google/mt5-small')

# tokenizer = MT5Tokenizer.from_pretrained("results/checkpoint-121500")  # это если мы хотим подгрузить веса модели которую уже обучали
# model = MT5ForConditionalGeneration.from_pretrained('results/checkpoint-121500')

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
max_target_tokens_count = 128
max_source_tokens_count = 512


def preprocess_function(examples):
    print(examples['text'][0])
    inputs = examples["text"]
    model_inputs = tokenizer(inputs, max_length=max_source_tokens_count, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=max_target_tokens_count, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
dataset = load_dataset('IlyaGusev/gazeta', revision="v2.0")
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
tokenized_dataset_train = tokenized_dataset['train'].select(range(6000))
tokenized_dataset_test= tokenized_dataset['test'].select(range(1000))

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Подробнее о метриках: https://clementbm.github.io/theory/2021/12/23/rouge-bleu-scores.html

In [ ]:
rouge = load_metric("rouge") #Recall-Oriented Understudy for Gisting Evaluation
bleu = evaluate.load("bleu") #Bilingual Evaluation Understudy

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = {}
    result_rouge = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    # Extract a few results
    result.update({key: value.mid.fmeasure * 100 for key, value in result_rouge.items()})

    result_bleu = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    # Extract a few results
    result["bleu"] = result_bleu["bleu"] * 100

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    result["char_len"] = np.mean([len(t) for t in decoded_preds])

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/NLP",
    evaluation_strategy="steps",
    eval_steps=25,
    logging_steps=25,
    learning_rate=4e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    fp16=False,
    predict_with_generate=True,
    generation_max_length=20,
    generation_num_beams=5
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=11, training_loss=3.6100602583451704, metrics={'train_runtime': 219.232, 'train_samples_per_second': 13.684, 'train_steps_per_second': 0.05, 'total_flos': 1488959129518080.0, 'train_loss': 3.6100602583451704, 'epoch': 0.94})

In [ ]:
trainer.evaluate()

{'eval_loss': 2.415264844894409,
 'eval_rouge1': 9.5556,
 'eval_rouge2': 2.4782,
 'eval_rougeL': 9.5118,
 'eval_rougeLsum': 9.5845,
 'eval_bleu': 0.2056,
 'eval_gen_len': 18.995,
 'eval_char_len': 67.513,
 'eval_runtime': 154.124,
 'eval_samples_per_second': 6.488,
 'eval_steps_per_second': 1.622,
 'epoch': 0.94}

In [ ]:
summarizer = pipeline("summarization", model = model)
summarizer("одно яблоко в день, и тебе не придется идти к врачу", min_length=5, max_length=20)

Exception: ignored

In [ ]:
summarizer = pipeline("summarization")
summarizer("одно яблоко в день, и тебе не придется идти к врачу", min_length=5, max_length=20)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

[{'summary_text': ' о япридет  тег'}]

In [ ]:
summarizer("i like apples, bannanas and oranges", min_length=5, max_length=11)

[{'summary_text': ' i like apples, bannanas and'}]

In [ ]:
import torch

def predict():
    article = str(input())
    inputs = tokenizer.encode("summarize: " + article, return_tensors="pt", max_length=100, truncation=True)
    return inputs.to('cuda')

inputs = predict()


model.to('cuda')

outputs = model.generate(
    inputs,
    max_length=40,
    min_length=5,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
)


print(tokenizer.decode(outputs[0]))


Пресс-секретарь президента России Дмитрий Песков заявил, что в Кремле не считают задержанных в РФ американцев заложниками. Об этом он рассказал в интервью NBC.  «Они под стражей, они не заложники», — заявил он.  Накануне президент России Владимир Путин на пресс-конференции заявил, что РФ не отказывается от обмена заключенными с США. По его словам, договоренности должны устраивать обе стороны.  Начальник пресс-службы Государственного департамента США Мэттью Миллер сказал, что первым шагом для возобновления переговоров между Москвой и Вашингтоном могло бы стать освобождение американских граждан, которые находятся под стражей на российской территории.
<pad> В Кремле не считают задержанных в РФ американцев заложниками. Об этом он рассказал в интервью NBC.</s>


In [ ]:
outputs = model.generate(
    predict(),
    max_length=40,
    min_length=5,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True)

print(tokenizer.decode(outputs[0]))

Дмитрия Афанаскина, отца школьницы Алины, устроившей стрельбу в гимназии в Брянске, выпустили из СИЗО. Меру пресечения ему сменили на запрет определенных действий. По версии следствия, он доводил дочь до самоубийства и небрежно хранил ружье, которое Алина принесла в школу в тубусе.
<pad>В Брянске обвиняют Дмитрия Афанаскина, отца школьницы Алины, устроившей стрельбу в гимназии в Брянске, устроившей стрель


In [ ]:
outputs = model.generate(
    predict(),
    max_length=40,
    min_length=5,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True)

print(tokenizer.decode(outputs[0]))

Тоня понадобилось 4 часа, чтобы понять как уменьшить набор данных. К счастью, у нее это получилось, и хоть модель работает недостаточно хорошо, она смогла исправить ошибку с неправильной длинной символов и спокойно пойдет спать.
<pad> Надежда Тоня смогла исправить ошибку с неправильной длинной символов. Тоня смогла исправить ошибку с неправильной длинной символов.</s>


In [ ]:
outputs = model.generate(
    predict(),
    max_length=40,
    min_length=5,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True)

print(tokenizer.decode(outputs[0]))

Пока Настя шла, она встретила медведя. Медведь сел в машину и сгорел.
<pad> В машину медведь сел в машину и сгорел медведь.</s>


In [ ]:
outputs = model.generate(
    predict(),
    max_length=512,
    min_length=5,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True)

print(tokenizer.decode(outputs[0]))

В павильоне «Роснефти» на форуме «Россия» открылись Дни судоверфи «Звезда», которая 17 декабря отметит свое 8-летие. Об этом сообщили в пресс-службе нефтяной компании.  На открытии дней «Звезды» гостям павильона показали фильм «Живые». Авторы картины сравнили суда с живыми организмами, которые рождаются на верфи, получают имена и становятся полноценными обитателями морских просторов. И все это благодаря инженерной мысли человека.
<pad> В павильоне «Роснефти» открылись Дни судоверфи «Звезда», которая отметит свое 8-летие. В павильоне «Роснефти» открылись Дни судоверфи «Звезда», которая отметит свое 8-летие.</s>
